In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

from keras import backend as K
from keras import losses
from keras.models import Sequential
from keras.layers import Activation,LeakyReLU
from keras.layers.core import Dense,Dropout
from keras.optimizers import Adam,SGD, Adagrad
from keras import metrics
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
data = pd.read_csv('final.csv',index_col=0)

In [3]:
y = data.iloc[:,-1]
X = data.drop(columns=['Rate','Year'])

##### Run this if one hot encoded month is needed 

In [17]:
enc = preprocessing.OneHotEncoder()
enc.fit(np.array(X['Month']).reshape(-1,1))

C:\Users\vatss\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [4]:
def adjustMonth(month):
    if month<4:
        return -1
    elif month<8:
        return 0
    else:
        return 1
X['Month']=X['Month'].apply(adjustMonth)

In [5]:
# Splittting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

##### Run this if one hot encoded month is needed 

In [19]:
months_train = enc.transform(np.array(X_train['Month']).reshape(-1,1))
months_test = enc.transform(np.array(X_test['Month']).reshape(-1,1))

In [20]:
months_train=months_train.todense()
months_test=months_test.todense()

In [21]:
X_train.drop(columns=['Month'],inplace=True)
X_train = X_train.to_numpy(copy=True)
X_test.drop(columns=['Month'],inplace=True)
X_test = X_test.to_numpy(copy=True)

C:\Users\vatss\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
X_train = np.concatenate((X_train,months_train),axis=1)
X_test = np.concatenate((X_test,months_test),axis=1)

In [6]:
X_train.shape

(153, 6)

##### Change the  following code block a little bit if using the one hot encoded months (remove iloc used for dataframes)

In [7]:
# scaling features
scaler = preprocessing.StandardScaler()
for i in range(X_train.shape[1]):
    colTrain = X_train.iloc[:,i]
#     colTrain = X_train[:,i]
    scaler.fit(np.array(colTrain).reshape(-1,1))
    X_train.iloc[:,i]=scaler.transform(np.array(colTrain).reshape(-1,1))
#     X_train[:,i]=scaler.transform(np.array(colTrain).reshape(-1,1))
    colTest = X_test.iloc[:,i]
#     colTest = X_test[:,i]
#     X_test[:,i]=scaler.transform(np.array(colTest).reshape(-1,1))    
    X_test.iloc[:,i]=scaler.transform(np.array(colTest).reshape(-1,1))

C:\Users\vatss\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\vatss\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\vatss\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [8]:
# scaling target
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

scaler.fit(y_train)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [9]:
X_train.shape

(153, 6)

##### Function for evaluation of model

#### 81,81,0.15,b=20,e=40,acc=64
#### 117,37, 0.17,b=20,e=45, acc=66
#### 131,41,0.2,ac==60,dropout(before last layer=0.2)
#### 76,46,0.21,acc=64 
#### 117,98,lr=0.17
#### 175,31,0.2
#### 85,31,0.215
#### 285,115,lr=0.19
#### 130,115,0.21
#### 290,155,0.2
#### 130,220,lr>0.215 maybe

In [166]:
def eval_model():
    model = Sequential()
    model.add(Dense(82,input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(),kernel_initializer='uniform',activation='relu'))
    model.add(Dense(12,kernel_regularizer=regularizers.l2(),kernel_initializer='uniform',activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(SGD(lr=0.17),loss='mean_absolute_error',metrics=['mae'])
    return model
model=eval_model()
model.fit(X_train,y_train,validation_split=0.2,epochs=40,batch_size=20,shuffle=True)
print(model.evaluate(X_test,y_test))

Train on 122 samples, validate on 31 samples
Epoch 1/40
122/122 [==============================] - 0s 2ms/step - loss: 0.8077 - mae: 0.7955 - val_loss: 0.8092 - val_mae: 0.7974
Epoch 2/40
122/122 [==============================] - 0s 0us/step - loss: 0.7409 - mae: 0.7293 - val_loss: 0.8333 - val_mae: 0.8219
Epoch 3/40
122/122 [==============================] - 0s 0us/step - loss: 0.7335 - mae: 0.7222 - val_loss: 0.8585 - val_mae: 0.8470
Epoch 4/40
122/122 [==============================] - 0s 128us/step - loss: 0.7520 - mae: 0.7407 - val_loss: 0.7710 - val_mae: 0.7599
Epoch 5/40
122/122 [==============================] - 0s 128us/step - loss: 0.7299 - mae: 0.7186 - val_loss: 0.8119 - val_mae: 0.8006
Epoch 6/40
122/122 [==============================] - 0s 128us/step - loss: 0.7194 - mae: 0.7082 - val_loss: 0.8469 - val_mae: 0.8355
Epoch 7/40
122/122 [==============================] - 0s 129us/step - loss: 0.7433 - mae: 0.7322 - val_loss: 0.7447 - val_mae: 0.7336
Epoch 8/40
122/122 [===

##### Hyperparametrization

In [178]:
def eval_model(n1=160,n2=170,lr=0.2):
    model = Sequential()
    model.add(Dense(n1,input_shape=(X_train.shape[1],),kernel_regularizer=regularizers.l2(),kernel_initializer='uniform',activation='relu'))
    model.add(Dense(n2,kernel_regularizer=regularizers.l2(),kernel_initializer='uniform',activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(SGD(lr=lr),loss='mean_squared_error',metrics=['mae'])
    return model

model = KerasRegressor(build_fn=eval_model, verbose=2,batch_size=24,epochs=45)
n1=[80,117,131,76,120,82,28,130,290,130]
n2=[80,56,46,41,37,5,12,27,220,155,115]
# n1=np.arange(80,300,5)
# n2=np.arange(80,300,5)
lr=[0.18,0.19,0.2,0.21,0.215,0.22,0.23]
param_grid = dict(n1=n1,n2=n2,lr=lr)
grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=-1, cv=4)
grid.fit(X_train, y_train)
print(grid.best_params_)
Model = grid.best_estimator_.model
print(Model.evaluate(X_test,y_test))

C:\Users\vatss\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/45
 - 0s - loss: 1.0200 - mae: 0.8533
Epoch 2/45
 - 0s - loss: 0.9870 - mae: 0.8193
Epoch 3/45
 - 0s - loss: 0.8858 - mae: 0.7655
Epoch 4/45
 - 0s - loss: 0.8590 - mae: 0.7269
Epoch 5/45
 - 0s - loss: 0.7631 - mae: 0.6682
Epoch 6/45
 - 0s - loss: 0.8394 - mae: 0.6749
Epoch 7/45
 - 0s - loss: 0.8274 - mae: 0.6820
Epoch 8/45
 - 0s - loss: 0.6525 - mae: 0.5544
Epoch 9/45
 - 0s - loss: 0.6924 - mae: 0.5746
Epoch 10/45
 - 0s - loss: 0.6304 - mae: 0.5418
Epoch 11/45
 - 0s - loss: 0.5697 - mae: 0.4992
Epoch 12/45
 - 0s - loss: 0.6659 - mae: 0.5278
Epoch 13/45
 - 0s - loss: 0.6739 - mae: 0.5409
Epoch 14/45
 - 0s - loss: 0.5987 - mae: 0.5042
Epoch 15/45
 - 0s - loss: 0.6180 - mae: 0.4987
Epoch 16/45
 - 0s - loss: 0.5505 - mae: 0.4637
Epoch 17/45
 - 0s - loss: 0.6534 - mae: 0.5370
Epoch 18/45
 - 0s - loss: 0.5261 - mae: 0.4381
Epoch 19/45
 - 0s - loss: 0.7444 - mae: 0.5846
Epoch 20/45
 - 0s - loss: 0.5704 - mae: 0.4820
Epoch 21/45
 - 0s - loss: 0.6298 - mae: 0.5246
Epoch 22/45
 - 0s - lo

##### Sklearn algorithm for prediction

In [24]:
# reg = SVR()
# SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
#     kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
# min_impurity_decrease=0.008,
reg = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=200,
                      max_features='auto', max_leaf_nodes=300,
                      min_impurity_decrease=0.008,
                      min_samples_leaf=2,
                      min_samples_split=4,
                      min_impurity_split=0.012,
                      n_estimators=100,
                      n_jobs=-1,random_state=0)

In [25]:
scores = cross_val_score(reg, X_train, y_train.ravel(), cv=5)
print(scores.mean())

0.5728572634640993


In [59]:
# y_pred = reg.predict(X_test)
reg.fit(X_train,y_train.ravel())
reg.score(X_test,y_test.ravel())

0.6292907059198538

In [155]:
from sklearn.neural_network import MLPRegressor
reg=MLPRegressor(validation_fraction=0.3,alpha=0.001,activation='relu',batch_size=24,hidden_layer_sizes=(28,56),max_iter=500,learning_rate_init=0.035)
reg.fit(X_train,y_train.ravel())
reg.score(X_test,y_test.ravel())

0.6370927447216156